In [17]:
import numpy as np
import matplotlib.pyplot as plt
import math

class Adaline():
    def __init__(self, 
                 num_of_weights, 
                 acceptable_error=0.35, 
                 max_iters=10000, 
                 learning_rate=0.01,
                 threshold=0.5,
                 max_weight_value=1):
        self.weights = np.random.rand(num_of_weights + 1, 1) * 2 * max_weight_value - max_weight_value
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.acceptable_error = acceptable_error
        self.epochs = 0
        self.cost = []
        self.threshold = threshold
        
    def error_cost(self, errors, n):
        return np.sum(([error**2 for error in errors])) / n
    
    def predict(self, x_arr):
        return x_arr @ self.weights[1:] + self.weights[0]
    
    def get_result(self, x_arr):
        return 1 if (x_arr @ self.weights[1:])[0][0] >= self.threshold else -1
        
    def train(self, x_train, y_train):
        for i in range(self.max_iters):
            self.epochs = self.epochs + 1
            errors = []
            costs = []
            for x_arr, y in zip(x_train, y_train):
                error = y - self.predict(x_arr)
                self.weights[1:] += self.learning_rate * error * x_arr.T
                self.weights[0] += self.learning_rate * error[0][0]
                
                costs.append(error)
                
            cost = self.error_cost(costs, len(x_train))
            if cost <= self.acceptable_error:
                break

In [18]:
np.random.seed(243)

training_x_arr = [np.array([[-1,-1]]),np.array([[-1,1]]),np.array([[1,-1]]),np.array([[1,1]])]
training_y_arr = [-1,-1,-1,1]

adaline = Adaline(num_of_weights=2, 
                  acceptable_error=0.3,
                  max_iters=1000,
                  learning_rate=0.01)

adaline.train(training_x_arr, training_y_arr)

print('Epochs:', adaline.epochs)
print('\nResults:')
for x_arr in training_x_arr:
    print(x_arr, ':', adaline.get_result(x_arr))

Epochs: 11

Results:
[[-1 -1]] : -1
[[-1  1]] : -1
[[ 1 -1]] : -1
[[1 1]] : 1


In [19]:
import pandas as pd

# 2.1 Starting weights testing

training_x_arr = [np.array([[-1,-1]]),np.array([[-1,1]]),np.array([[1,-1]]),np.array([[1,1]])]
training_y_arr = [-1,-1,-1,1]

max_weight_values = [1, 0.8, 0.6, 0.4, 0.2, 0.1]
epoch_values = [0, 0, 0, 0, 0, 0]
iterations = 10

for j in range(iterations):
    for i in range(len(max_weight_values)):
        adaline = Adaline(num_of_weights=2, max_weight_value=max_weight_values[i], threshold=0.5)
        adaline.train(training_x_arr, training_y_arr)
        epoch_values[i] += adaline.epochs
        
df = pd.DataFrame(columns=['Weights', 'Epochs'])
for i in range(len(max_weight_values)):
    df = df.append({'Weights': f'[{-max_weight_values[i]}, {max_weight_values[i]}]', 'Epochs': epoch_values[i] / iterations}, ignore_index=True)
    
display(df)
df.to_csv('./Results/adaline_weights_testing.csv', index=False)

,Weights,Epochs
0,"[-1, 1]",39.7
1,"[-0.8, 0.8]",35.3
2,"[-0.6, 0.6]",26.1
3,"[-0.4, 0.4]",29.4
4,"[-0.2, 0.2]",27.0
5,"[-0.1, 0.1]",26.5


In [20]:
# 2.2 Learning rate testing

training_x_arr = [np.array([[-1,-1]]),np.array([[-1,1]]),np.array([[1,-1]]),np.array([[1,1]])]
training_y_arr = [-1,-1,-1,1]

df = pd.DataFrame(columns=['Learning rate', 'Epochs'])
learning_rates = [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1]
iterations = 10
epochs = [0] * len(learning_rates)

for i in range(iterations):
    for j, learning_rate in enumerate(learning_rates):
        adaline = Adaline(num_of_weights=2, learning_rate=learning_rate, threshold=0.5, max_iters=1000)
        adaline.train(training_x_arr, training_y_arr)
        epochs[j] += adaline.epochs
        
epochs = [x / iterations for x in epochs]
        
for learning_rate, epoch in zip(learning_rates, epochs):
    df = df.append({'Learning rate': learning_rate, 'Epochs': epoch}, ignore_index=True)
    
display(df)
df.to_csv('./Results/adaline_learning_rate_testing.csv', index=False)

,Learning rate,Epochs
0,0.001,276.4
1,0.005,60.0
2,0.010,28.9
3,0.015,20.5
4,0.020,18.4
5,0.025,13.8
6,0.030,12.1
7,0.035,10.9
8,0.040,10.2
9,0.045,9.5


In [21]:
# 2.3 Acceptable error testing

training_x_arr = [np.array([[-1,-1]]),np.array([[-1,1]]),np.array([[1,-1]]),np.array([[1,1]])]
training_y_arr = [-1,-1,-1,1]

df = pd.DataFrame(columns=['Error', 'Epochs'])
acceptable_errors = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
iterations = 10

epochs = [0] * len(acceptable_errors)

for i in range(iterations):
    for j, acceptable_error in enumerate(acceptable_errors):
        adaline = Adaline(num_of_weights=2, acceptable_error=acceptable_error, threshold=0.5, max_iters=1000)
        adaline.train(training_x_arr, training_y_arr)
        epochs[j] += adaline.epochs
        
epochs = [x / iterations for x in epochs]
        
for acceptable_error, epoch in zip(acceptable_errors, epochs):
    df = df.append({'Error': acceptable_error, 'Epochs': epoch}, ignore_index=True)
    
display(df)
df.to_csv('./Results/adaline_error_testing.csv', index=False)

,Error,Epochs
0,0.30,40.4
1,0.35,35.8
2,0.40,29.9
3,0.45,19.3
4,0.50,26.6
5,0.55,21.5
6,0.60,20.1
7,0.65,18.8
8,0.70,18.5
9,0.75,12.0
